In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
from sqlalchemy import create_engine
import pymysql
%matplotlib inline

In [2]:
anime = pd.read_csv('anime.csv')
ratings = pd.read_csv('rating.csv')

In [3]:
ratings.rating = ratings.rating.replace({-1: np.nan})
ratings.head()

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN


In [4]:
anime_tv = anime[anime['type']=='TV']
anime_tv.head()
merged = ratings.merge(anime_tv, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])
merged.rename(columns = {'rating_user':'user_rating'}, inplace = True)

In [5]:
merged.head()

,user_id,anime_id,user_rating,name,genre,type,episodes,rating,members
0,1,20,NaN,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
1,3,20,8.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
2,5,20,6.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
3,6,20,NaN,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
4,10,20,NaN,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297


In [6]:
merged=merged[['user_id', 'anime_id', 'user_rating']]
merged_sub= merged[merged.user_id <= 10000]
merged_sub.head()

,user_id,anime_id,user_rating
0,1,20,NaN
1,3,20,8.0
2,5,20,6.0
3,6,20,NaN
4,10,20,NaN


Create a pivot table of users on one axies and animate names along the other. 
Pivot table helps us in defining the similarity between users

In [7]:
piv = merged_sub.pivot_table(index=['user_id'], columns=['anime_id'], values='user_rating')
print(piv.head)

<bound method NDFrame.head of anime_id  1      6      7      8      15     16     17     18     19     \
user_id                                                                   
1           NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2           NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3           NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5           NaN    8.0    NaN    NaN    6.0    NaN    6.0    6.0    NaN   
7           NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
8           NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
9           NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
10          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
11          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
12          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
14          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    Na

In [8]:
#Normalize the values
#Drop all columns containing only zeros representing users who did not rate
piv.fillna(0, inplace=True)
piv= piv.T
piv = piv.loc[:, (piv != 0).any(axis=0)]
piv.iloc[0,:].max()
piv.head()

user_id,1,2,3,5,7,8,9,10,11,12,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
anime_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
#change piv into sparse matrix
#piv_sparse = sp.sparse.csr_matrix(piv_norm.values)

In [10]:
#get item_similarity and user_similarity
#item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv.T)

In [11]:
print(user_similarity)

[[1.         0.         0.11653496 ... 0.21945215 0.09756098 0.10243158]
 [0.         1.         0.15537994 ... 0.         0.         0.        ]
 [0.11653496 0.15537994 1.         ... 0.16185403 0.17470769 0.09231171]
 ...
 [0.21945215 0.         0.16185403 ... 1.         0.11133182 0.10190403]
 [0.09756098 0.         0.17470769 ... 0.11133182 1.         0.15989319]
 [0.10243158 0.         0.09231171 ... 0.10190403 0.15989319 1.        ]]


In [12]:
s = np.argsort(user_similarity,axis=1)
s = s[:,-11:]
print(len(s[1]))
s = s[:,::-1]
print(s)
a = piv.columns[s].tolist()
print(a[2])
 

11
[[   0 1660 9222 ...  754 8638 1053]
 [   1  626 2580 ... 2123 7472  540]
 [   2 1793 3140 ... 3614 8872 8880]
 ...
 [9384 8912  411 ... 8217 3618 8313]
 [9385 8831 3187 ... 9131 8062 3777]
 [9386 1995 3721 ... 9380 5584   40]]
[3, 1918, 3376, 9615, 3890, 1554, 1354, 9491, 3881, 9459, 9467]


In [13]:
piv = piv.T
sim_user = pd.DataFrame(a,columns=['sim'+str(i) for i in range(len(a[0]))])
sim_user.head()

,sim0,sim1,sim2,sim3,sim4,sim5,sim6,sim7,sim8,sim9,sim10
0,1,1776,9825,9775,3606,9450,8864,4364,809,9204,1126
1,2,671,2773,3282,4185,1810,1187,4308,2277,7957,583
2,3,1918,3376,9615,3890,1554,1354,9491,3881,9459,9467
3,5,5255,8231,7816,662,4306,1938,8115,3275,9752,1176
4,7,1287,3476,6836,4280,1103,1044,1929,4326,4650,2300


In [14]:
sim_user.sim0.values

array([    1,     2,     3, ...,  9998,  9999, 10000], dtype=int64)

In [15]:
def top_users(user_id):
    if user_id not in sim_user.sim0.values:
        return ('No data available on user {}'.format(user_id))
    #sim_values = user_sim_df.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:11]
    idx = sim_user[(sim_user.sim0 == user_id)].index
    s_users = sim_user.loc[idx,:]
    return s_users.values.tolist()[0][1:]

In [16]:
print(top_users(1))

[1776, 9825, 9775, 3606, 9450, 8864, 4364, 809, 9204, 1126]


In [17]:
piv.head()

anime_id,1,6,7,8,15,16,17,18,19,20,...,33028,33037,33046,33113,33222,33241,33274,33341,33394,33421
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,8.0,0.0,0.0,6.0,0.0,6.0,6.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0


In [18]:
piv = piv.T

In [19]:
piv.head()

user_id,1,2,3,5,7,8,9,10,11,12,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
anime_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
best = []
for i in piv.columns:
    max_score = piv.loc[:,i].max()
    best.append(piv[piv.loc[:, i] == max_score].index.tolist())
    

In [21]:
best[0]

[8074, 11617, 11757, 15451]

In [22]:
max_len = max([len(i) for i in best])
print(max_len)

368


In [23]:
for i in best:
    i.extend(0 for _ in range(max_len-len(i)))

In [24]:
best_df = pd.DataFrame(best)

In [25]:
best_df['id'] = piv.columns

In [26]:
best_df.head()

,0,1,2,3,4,5,6,7,8,9,...,359,360,361,362,363,364,365,366,367,id
0,8074,11617,11757,15451,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,11771,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,813,1535,5114,9989,11771,14513,16498,16894,18115,24415,...,0,0,0,0,0,0,0,0,0,3
3,245,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
4,30,4896,14653,16982,19815,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7


In [30]:
from sqlalchemy import create_engine
engine = create_engine('<Database sqlalchemy>')
sim_user.to_sql('user_sim', con=engine, index=False)
best_df.to_sql('favorit', con=engine, index=False)

In [28]:
def similar_user_recs(user):
    if user not in sim_user.sim0.values:
        return('No data available on user {}'.format(user))
    sim_users = top_users(user)
    best = []
    most_common = {}
    for i in sim_users:
        max_score = piv.loc[:, i].max()
        best.append(piv[piv.loc[:, i]==max_score].index.tolist())
    for i in range(len(best)):
        for j in best[i]:
            if j in most_common:
                most_common[j] += 1
            else:
                most_common[j] = 1
    sorted_list = sorted(most_common.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_list[:5],[sorted_list[i][0] for i in range(5)]

In [29]:
_,list = similar_user_recs(3)
print(list)

[5114, 1535, 31043, 16498, 1575]
